In [1]:
import os
import re
import rasterio
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.geometry import mapping
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [2]:
HIMA_PATH = "DATA_SV/Hima"
ERA5_PATH = "DATA_SV/ERA5"
RADAR_PATH = "DATA_SV/Precipitation/Radar"

OUTPUT_X = "csv_data/tri_an_thanh_hoa/x.npy"
OUTPUT_Y = "csv_data/tri_an_thanh_hoa/y.npy"

OUTPUT_RADAR_CSV = "csv_data/tri_an_thanh_hoa/RADAR_CSV.csv"
OUTPUT_ERA5_CSV = "csv_data/tri_an_thanh_hoa/ERA5_CSV.csv"
OUTPUT_HIMA_CSV = "csv_data/tri_an_thanh_hoa/HIMA_CSV.csv"

selected_features = ['B04B', 'B10B', 'B11B', 'B16B', 'IRB',
                     'CAPE', 'R850', 'TCWV', 'U850', 'I2B', 'TCLW', 'TCW']


In [3]:

# =========================================================
# 2) TRÍCH DATETIME TỪ FILENAME
# =========================================================
def extract_datetime_from_filename(path):
    filename = os.path.basename(path)

    # Kiểu 1: CAPE_20190401000000.tif
    m14 = re.search(r"(\d{14})", filename)
    if m14:
        return pd.to_datetime(m14.group(1), format="%Y%m%d%H%M%S", errors="coerce")

    # Kiểu 2: B04B_20190401.Z0000_TB.tif
    m_date = re.search(r"(\d{8})", filename)
    m_z = re.search(r"Z(\d{4})", filename)

    if m_date:
        date = m_date.group(1)
        if m_z:
            return pd.to_datetime(date + m_z.group(1), format="%Y%m%d%H%M", errors="coerce")
        return pd.to_datetime(date, format="%Y%m%d", errors="coerce")

    return pd.NaT


# =========================================================
# 3) LIST FILE
# =========================================================
def list_all_files(root):
    out = []
    for dp, _, files in os.walk(root):
        for f in files:
            if f.endswith(".tif") or f.endswith(".TIF"):
                out.append(os.path.join(dp, f))
    return out


def fill_nodata_minus9999(arr):
    """
    Fill toàn bộ NaN / +Inf / -Inf thành -9999
    Nhanh, an toàn cho ML (XGBoost / CatBoost / LGBM)
    """
    arr = arr.astype(np.float32, copy=False)
    mask = np.isnan(arr) | np.isinf(arr)
    if mask.any():
        arr[mask] = -9999.0
    return arr


def get_variable_name(path):
    """
    Lấy tên biến từ filename.
    VD: 'B04B_2019....tif' -> trả về 'B04B'
    VD: 'CAPE_2019....tif' -> trả về 'CAPE'
    """
    filename = os.path.basename(path)
    return filename.split('_')[0]

 Lấy các timestamp hợp lệ
 (tồn tại ở cả 3 folder ERA5, HIMA, RADAR và trong các band đã chọn)

In [4]:

def get_final_common_timestamps(list_folders, selected_features):
    """
    Chiến thuật:
    1. Quét TOÀN BỘ file trong 3 folder (bất kể tên gì) để tìm giao điểm thời gian lớn nhất (cái 2337 mốc kia).
    2. Sau khi có bộ khung thời gian chung, đi kiểm tra lại xem những giờ đó có đủ các feature B04B, CAPE... không.
    """

    # --- BƯỚC 1: QUÉT TẤT CẢ ĐỂ LẤY TIMESTAMP CHUNG (như bạn đã làm ra 2337) ---
    print(f"-> [B1] Quét toàn bộ 3 folder để tìm giao điểm thời gian...")
    list_sets = []

    # Biến lưu trữ map: {timestamp: {variable_name: filepath}}
    # Để tí nữa check feature cho nhanh, không cần scan lại
    mega_map = {}

    for folder in list_folders:
        current_ts_set = set()
        print(f"   Scanning: {folder}...")

        for root, _, files in os.walk(folder):
            for f in files:
                if f.lower().endswith(('.tif', '.tiff')):
                    fpath = os.path.join(root, f)
                    ts = extract_datetime_from_filename(fpath)

                    if pd.notna(ts):
                        current_ts_set.add(ts)

                        # Lưu lại thông tin file để dùng cho Bước 2
                        # Lấy prefix làm tên biến (B04B, Radar, CAPE...)
                        var_name = f.split('_')[0]

                        # Mapping tên cho Radar (Quan trọng!)
                        if var_name == 'Radar' or f.startswith('2019') or f.startswith('2020'):
                            var_name = 'y'

                        if ts not in mega_map: mega_map[ts] = set()
                        mega_map[ts].add(var_name)

        list_sets.append(current_ts_set)

    # Giao nhau giữa 3 folder
    if not list_sets: return []
    common_ts = set.intersection(*list_sets)
    sorted_common = sorted(list(common_ts))

    print(f"-> Đã tìm thấy {len(sorted_common)} mốc thời gian chung (Raw Intersection).")

    # --- BƯỚC 2: LỌC LẠI THEO FEATURE ---
    # Bây giờ ta chỉ giữ lại những mốc thời gian mà tại đó có ĐỦ các feature yêu cầu

    print(f"-> [B2] Kiểm tra tính đầy đủ của Feature...")
    print(f"   Yêu cầu: {selected_features} + ['y']")

    final_valid_ts = []

    # Input feature + y
    required_set = set(selected_features + ['y'])

    for ts in sorted_common:
        # Lấy danh sách các biến CÓ MẶT tại thời điểm ts
        vars_at_ts = mega_map.get(ts, set())

        # Kiểm tra xem có chứa đủ bộ required không
        # Lưu ý: vars_at_ts có thể chứa nhiều biến rác khác, ta chỉ quan tâm nó có chứa đủ bộ required không thôi
        if required_set.issubset(vars_at_ts):
            final_valid_ts.append(ts)

    print("\n" + "=" * 40)
    print(f"✅ KẾT QUẢ CUỐI CÙNG: {len(final_valid_ts)} mốc thời gian ĐỦ DỮ LIỆU.")
    print("=" * 40)

    if len(final_valid_ts) == 0 and len(sorted_common) > 0:
        print("⚠️ CẢNH BÁO: Có giao điểm thời gian nhưng bị thiếu Feature!")
        # Debug thử 1 mẫu
        sample_ts = sorted_common[0]
        print(f"   Tại {sample_ts} có các biến: {mega_map[sample_ts]}")
        print(f"   Thiếu: {required_set - mega_map[sample_ts]}")

    return final_valid_ts



In [5]:
FOLDERS = [HIMA_PATH, ERA5_PATH, RADAR_PATH]

FINAL_TIMESTAMPS = get_final_common_timestamps(FOLDERS, selected_features)


-> [B1] Quét toàn bộ 3 folder để tìm giao điểm thời gian...
   Scanning: DATA_SV/Hima...
   Scanning: DATA_SV/ERA5...
   Scanning: DATA_SV/Precipitation/Radar...
-> Đã tìm thấy 2337 mốc thời gian chung (Raw Intersection).
-> [B2] Kiểm tra tính đầy đủ của Feature...
   Yêu cầu: ['B04B', 'B10B', 'B11B', 'B16B', 'IRB', 'CAPE', 'R850', 'TCWV', 'U850', 'I2B', 'TCLW', 'TCW'] + ['y']

✅ KẾT QUẢ CUỐI CÙNG: 1223 mốc thời gian ĐỦ DỮ LIỆU.


In [6]:
print(FINAL_TIMESTAMPS[:10])

[Timestamp('2019-04-01 00:00:00'), Timestamp('2019-04-01 01:00:00'), Timestamp('2019-04-01 02:00:00'), Timestamp('2019-04-01 03:00:00'), Timestamp('2019-04-01 04:00:00'), Timestamp('2019-04-01 05:00:00'), Timestamp('2019-04-01 06:00:00'), Timestamp('2019-04-01 07:00:00'), Timestamp('2019-04-01 08:00:00'), Timestamp('2019-04-01 09:00:00')]


## Tải dữ liệu ranh giới tỉnh Thanh Hóa


In [7]:
shp_path = "gadm41_VNM_shp"
vnm_gdf = gpd.read_file(shp_path)

th_gdf = vnm_gdf[vnm_gdf['VARNAME_1'] == 'Thanh Hoa']

th_union = th_gdf.geometry.union_all()
th_crs = th_gdf.crs

# I. Xử lí Load ground Truth

## 1.Lấy các pixel chuẩn thuộc tỉnh Thanh Hóa

xử lí cho Y

In [8]:
def extract_ThanhHoa_pixels(path, root):
    try:
        with rasterio.open(path) as src:
            data = src.read(1).astype(float)
            nodata = src.nodata
            transform = src.transform
            src_crs = src.crs

        # --- chuẩn hóa NODATA ---
        data[data == nodata] = np.nan
        data[data == -9999] = np.nan
        data[np.isinf(data)] = np.nan

        # --- fill nodata bằng nearest ---
        data = fill_nodata_minus9999(data)

        # --- reproject Hà Tĩnh sang CRS raster ---
        if src_crs != th_crs:
            geom = th_gdf.to_crs(src_crs).geometry.union_all()
        else:
            geom = th_union

        # --- tạo mask pixel thuộc Thanh Hoas ---
        mask = geometry_mask(
            [mapping(geom)],
            invert=True,
            out_shape=data.shape,
            transform=transform
        )

        rows, cols = np.where(mask)
        vals = data[rows, cols]

        # --- nếu không có pixel trong thanh hoa ---
        if len(rows) == 0:
            cr = data.shape[0] // 2
            cc = data.shape[1] // 2
            rows, cols = np.array([cr]), np.array([cc])
            vals = np.array([data[cr, cc]])

        # --- timestamp ---
        ts = extract_datetime_from_filename(path)

        # --- variable ---
        rel = os.path.relpath(path, root)
        var = rel.split(os.sep)[0]

        return pd.DataFrame({
            "variable": var,
            "timestamp": ts,
            "row": rows,
            "col": cols,
            "value": vals
        })

    except Exception as e:
        print("ERROR:", path, e)
        return pd.DataFrame({
            "variable": "unknown",
            "timestamp": pd.NaT,
            "row": [0],
            "col": [0],
            "value": [np.nan],
        })

xử lí cho X

In [18]:
def extract_ThanhHoa_bbox_all_pixels(path, root):
    try:
        with rasterio.open(path) as src:
            data = src.read(1).astype(float)
            nodata = src.nodata
            transform = src.transform
            src_crs = src.crs

        # --- 1. Xử lý Nodata (vẫn giữ nguyên logic của bạn) ---
        # Lưu ý: Nếu pixel bên ngoài tỉnh là nodata của ảnh gốc, nó sẽ thành nan ở đây
        if nodata is not None:
             data[data == nodata] = np.nan
        data[data == -9999] = np.nan
        data[np.isinf(data)] = np.nan

        # Hàm này của bạn, giả định đã được define ở ngoài
        data = fill_nodata_minus9999(data)

        # --- 2. Chuẩn bị Geometry ---
        # Lưu ý: th_crs, th_gdf, th_union phải là biến global hoặc truyền vào
        if src_crs != th_crs:
            geom = th_gdf.to_crs(src_crs).geometry.union_all()
        else:
            geom = th_union

        # --- BƯỚC 1: Loose bbox (Khung bao quát) ---
        bbox = geom.bounds
        min_row_loose, min_col_loose = rasterio.transform.rowcol(transform, bbox[0], bbox[3])
        max_row_loose, max_col_loose = rasterio.transform.rowcol(transform, bbox[2], bbox[1])

        # Kẹp index trong phạm vi ảnh
        min_row_loose = int(max(0, min_row_loose))
        min_col_loose = int(max(0, min_col_loose))
        max_row_loose = int(min(data.shape[0] - 1, max_row_loose))
        max_col_loose = int(min(data.shape[1] - 1, max_col_loose))

        # --- BƯỚC 2: Tạo Mask trong Loose bbox ---
        height_loose = max_row_loose - min_row_loose + 1
        width_loose = max_col_loose - min_col_loose + 1

        # Tạo transform cho window con
        window_transform = rasterio.windows.transform(
            rasterio.windows.Window(min_col_loose, min_row_loose, width_loose, height_loose),
            transform
        )

        # Mask: True = Nằm ngoài, False = Nằm trong (Mặc định rasterio)
        # invert=True -> Mask sẽ trả về True nếu pixel nằm TRONG shape
        mask_subset = geometry_mask(
            [mapping(geom)],
            invert=True,
            out_shape=(height_loose, width_loose),
            transform=window_transform
        )

        # --- BƯỚC 3: Tight bbox (Khung bao sát sạt pixel thật) ---
        # Tìm các hàng/cột có chứa ít nhất 1 pixel thuộc tỉnh
        valid_rows, valid_cols = np.where(mask_subset)

        if len(valid_rows) == 0:
            print(f"WARNING: No pixel inside geometry: {path}")
            return pd.DataFrame()

        trim_min_r, trim_max_r = valid_rows.min(), valid_rows.max()
        trim_min_c, trim_max_c = valid_cols.min(), valid_cols.max()

        # Tính lại toạ độ toàn cục (Global indices)
        min_row = min_row_loose + trim_min_r
        max_row = min_row_loose + trim_max_r
        min_col = min_col_loose + trim_min_c
        max_col = min_col_loose + trim_max_c

        # --- BƯỚC 4: Lấy dữ liệu ---
        # Tạo lưới toạ độ cho Tight Bbox
        rows = np.arange(min_row, max_row + 1)
        cols = np.arange(min_col, max_col + 1)

        row_grid, col_grid = np.meshgrid(rows, cols, indexing='ij')
        rows_flat = row_grid.flatten()
        cols_flat = col_grid.flatten()

        # Lấy giá trị từ dữ liệu gốc
        # LƯU Ý: Ở đây 'vals' lấy trực tiếp từ 'data', nên nó giữ nguyên giá trị gốc
        vals = data[rows_flat, cols_flat]

        # Cắt mask tương ứng với Tight Bbox để biết điểm nào thuộc tỉnh, điểm nào không
        mask_tight = mask_subset[
            trim_min_r:trim_max_r + 1,
            trim_min_c:trim_max_c + 1
        ].flatten()

        # --- ĐÃ XÓA ĐOẠN GÁN -1 Ở ĐÂY ---
        # Trước đây: vals[~mask_tight] = -1  <-- XÓA DÒNG NÀY

        ts = extract_datetime_from_filename(path)
        rel = os.path.relpath(path, root)
        var = rel.split(os.sep)[0]

        return pd.DataFrame({
            "variable": var,
            "timestamp": ts,
            "row": rows_flat,
            "col": cols_flat,
            "value": vals,           # Giá trị gốc (có thể là NaN nếu là background của ảnh Tif)
            "is_inside_shape": mask_tight # True: thuộc tỉnh, False: thuộc hcn bao quanh nhưng ngoài tỉnh
        })

    except Exception as e:
        print("ERROR:", path, e)
        # In traceback để dễ debug hơn nếu cần
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

TIF THANH HÓA => CSV

In [19]:
def tif2csv_ThanhHoa(type):
    output_file = OUTPUT_ERA5_CSV
    root = ERA5_PATH

    if type == "radar":
        output_file = OUTPUT_RADAR_CSV
        root = RADAR_PATH
    elif type == "hima":
        output_file = OUTPUT_HIMA_CSV
        root = HIMA_PATH

    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    files = list_all_files(root)
    print("Tổng file tìm thấy:", len(files))

    if os.path.exists(output_file):
        os.remove(output_file)

    if type == "era5" or type == "hima":
        func = partial(extract_ThanhHoa_bbox_all_pixels, root=root)
    else:
        func = partial(extract_ThanhHoa_pixels, root=root)

    results = []

    with ThreadPoolExecutor(max_workers=10) as pool:
        futures = [pool.submit(func, f) for f in files]

        for f in tqdm(as_completed(futures), total=len(futures), desc="Process"):
            try:
                df = f.result()
                if df is not None and not df.empty:
                    results.append(df)
            except Exception as e:
                print("Thread error:", e)

    if results:
        final = pd.concat(results, ignore_index=True)
        final.to_csv(output_file, index=False)
        print("DONE! Tổng pixel =", len(final))
    else:
        print("Không có data.")



In [ ]:
tif2csv_ThanhHoa("radar")

In [20]:
tif2csv_ThanhHoa("era5")
tif2csv_ThanhHoa("hima")

Tổng file tìm thấy: 58560


Process: 100%|██████████| 58560/58560 [10:39<00:00, 91.57it/s] 


DONE! Tổng pixel = 83623680
Tổng file tìm thấy: 33064


Process: 100%|██████████| 33064/33064 [04:15<00:00, 129.58it/s]


DONE! Tổng pixel = 47215392


# chuyển dữ liệu từ file.csv => numpy


In [21]:
def create_x_y_selected_features(list_path):

    # 1) Đọc CSV và gộp
    dfs = []
    print("[B1] Đọc CSV...")
    for p in tqdm(list_path, desc="Đọc file CSV"):
        # Mẹo: Xác định dtype ngay lúc đọc để tiết kiệm bộ nhớ nếu file lớn
        df = pd.read_csv(p)
        df["variable"] = df["variable"].astype(str)
        # Gán nhãn y
        df.loc[df["variable"].isin(['2019', '2020']), "variable"] = 'y'
        dfs.append(df)

    df_all = pd.concat(dfs, ignore_index=True)

    # 2) Min/Max row/col → tạo hình chữ nhật
    min_row, max_row = df_all["row"].min(), df_all["row"].max()
    min_col, max_col = df_all["col"].min(), df_all["col"].max()
    n_row = int(max_row - min_row + 1)
    n_col = int(max_col - min_col + 1)

    # 3) Xác định danh sách band cần thiết (QUAN TRỌNG: Phải Sort để cố định thứ tự)
    required_bands_list = sorted(list(set(selected_features + ['y'])))
    required_bands_set = set(required_bands_list)
    print(f"-> Số lượng band bắt buộc: {len(required_bands_list)}")

    # 4) Lọc Timestamp hợp lệ (TỐI ƯU HÓA TỐC ĐỘ)
    print("[B2] Lọc timestamp đầy đủ (Vectorized Check)...")

    # Chỉ giữ lại các dòng thuộc các variable quan tâm để đếm cho nhanh
    df_check = df_all[df_all["variable"].isin(required_bands_set)]

    # Đếm số lượng variable unique trong mỗi timestamp
    # Nếu timestamp T1 có đủ 13 variable -> count sẽ là 13
    ts_counts = df_check.groupby("timestamp")["variable"].nunique()

    # Lấy ra các timestamp có số lượng variable bằng đúng số lượng yêu cầu
    valid_ts_index = ts_counts[ts_counts == len(required_bands_set)].index
    ts_valid = sorted(list(valid_ts_index))

    print(f"-> Tìm thấy {len(ts_valid)} timestamp hợp lệ.")

    # 5) Chuẩn bị dữ liệu để đổ vào Tensor
    print("[B3] Chuẩn bị index và Tensor...")

    # Tạo mapping index (Dictionary comprehension)
    t_to_idx = {t: i for i, t in enumerate(ts_valid)}
    b_to_idx = {b: i for i, b in enumerate(required_bands_list)} # Dùng list đã sort

    # Lọc dữ liệu chính thức:
    # - Chỉ lấy timestamp hợp lệ
    # - Chỉ lấy variable nằm trong required_bands (Bước này sửa lỗi Index Float)
    df_valid = df_all[
        (df_all["timestamp"].isin(ts_valid)) &
        (df_all["variable"].isin(required_bands_set))
    ].copy()

    # Map sang index (Ép kiểu int rõ ràng để tránh lỗi)
    df_valid["t_idx"] = df_valid["timestamp"].map(t_to_idx).astype(int)
    df_valid["b_idx"] = df_valid["variable"].map(b_to_idx).astype(int)
    df_valid["r_idx"] = (df_valid["row"] - min_row).astype(int)
    df_valid["c_idx"] = (df_valid["col"] - min_col).astype(int)

    # 6) Đổ dữ liệu vào Tensor (Vectorized - Không cần vòng lặp)
    print("[B4] Đổ dữ liệu vào Tensor...")

    tensor = np.full(
        (len(ts_valid), len(required_bands_list), n_row, n_col),
        -1,
        dtype=float
    )

    # Numpy Advanced Indexing: Nhanh hơn loop rất nhiều
    tensor[df_valid["t_idx"].values,
           df_valid["b_idx"].values,
           df_valid["r_idx"].values,
           df_valid["c_idx"].values] = df_valid["value"].values

    # 7) Tách X và y
    y_idx = b_to_idx['y']
    # Lấy mảng X indices: loại bỏ index của y
    x_indices = [i for i, b in enumerate(required_bands_list) if b != 'y']

    y = tensor[:, [y_idx], :, :]
    x = tensor[:, x_indices, :, :]

    return x, y, ts_valid, required_bands_list, (min_row, max_row), (min_col, max_col)

In [22]:
def luuTensor():
    list_file = [
        OUTPUT_HIMA_CSV,
        OUTPUT_ERA5_CSV,
        OUTPUT_RADAR_CSV
    ]

    x, y, timestamps, x_bands, row_range, col_range = create_x_y_selected_features(list_file)

    print("[B4] Lưu tensor...")
    np.save(OUTPUT_X, x)
    np.save(OUTPUT_Y, np.squeeze(y, axis=1))

luuTensor()

[B1] Đọc CSV...


Đọc file CSV: 100%|██████████| 3/3 [00:54<00:00, 18.27s/it]


-> Số lượng band bắt buộc: 13
[B2] Lọc timestamp đầy đủ (Vectorized Check)...
-> Tìm thấy 1223 timestamp hợp lệ.
[B3] Chuẩn bị index và Tensor...
[B4] Đổ dữ liệu vào Tensor...
[B4] Lưu tensor...


In [12]:
# def check_data_stats(csv_path):
#     print(f"Dang doc file: {csv_path} ...")
#
#     try:
#         # Load file CSV
#         df = pd.read_csv(csv_path)
#
#         # Lấy cột giá trị (thường tên là 'value')
#         if 'value' not in df.columns:
#             print("Lỗi: Không tìm thấy cột 'value' trong CSV.")
#             print("Các cột hiện có:", df.columns)
#             return
#
#         total_pixels = len(df)
#         if total_pixels == 0:
#             print("File rỗng! Không có dữ liệu.")
#             return
#
#         # 1. Đếm số lượng số 0
#         zero_count = (df['value'] == 0).sum()
#         zero_ratio = (zero_count / total_pixels) * 100
#
#         # 2. Đếm số lượng NaN (dữ liệu lỗi/thiếu)
#         nan_count = df['value'].isna().sum()
#         nan_ratio = (nan_count / total_pixels) * 100
#
#         # 3. Tìm giá trị lớn nhất (để xem có mưa thật không)
#         max_val = df['value'].max()
#         min_val = df['value'].min()
#         mean_val = df['value'].mean()
#
#         # --- IN KẾT QUẢ ---
#         print("-" * 30)
#         print("BÁO CÁO THỐNG KÊ DỮ LIỆU")
#         print("-" * 30)
#         print(f"Tổng số dòng (pixels): {total_pixels:,}")
#         print(f"Giá trị MAX (Mưa lớn nhất): {max_val}")
#         print(f"Giá trị MIN: {min_val}")
#         print(f"Giá trị Trung bình: {mean_val:.4f}")
#         print("-" * 30)
#         print(f"Số lượng điểm = 0: {zero_count:,} ({zero_ratio:.2f}%)")
#         print(f"Số lượng điểm NaN: {nan_count:,} ({nan_ratio:.2f}%)")
#         print("-" * 30)
#
#         # --- ĐÁNH GIÁ SƠ BỘ ---
#         if zero_ratio == 100:
#             print("⚠️ CẢNH BÁO: Dữ liệu toàn số 0. Kiểm tra lại code trích xuất!")
#         elif zero_ratio > 90:
#             print("ℹ️ LƯU Ý: Tỉ lệ số 0 rất cao (>90%). Điều này BÌNH THƯỜNG nếu là dữ liệu mưa (mưa thưa).")
#         elif max_val == 0:
#             print("⚠️ CẢNH BÁO: Max value = 0. Có vẻ như không có chút mưa nào được ghi nhận.")
#         else:
#             print("✅ Dữ liệu trông có vẻ hợp lý.")
#
#     except Exception as e:
#         print(f"Có lỗi khi đọc file: {e}")
#
#
# # --- CÁCH SỬ DỤNG ---
# # Thay đường dẫn file CSV của bạn vào đây
# csv_file_path = OUTPUT_Y_CSV  # Biến này bạn đã khai báo ở đoạn code trước
# check_data_stats(csv_file_path)

CSV PIXEL THANH HÓA => NUMPY

In [13]:
def csv2numpy_thanhhoa(list_path):

    # 1) Đọc CSV và gộp
    dfs = []
    print("[B1] Đọc CSV...")
    for p in tqdm(list_path, desc="Đọc file CSV"):
        # Mẹo: Xác định dtype ngay lúc đọc để tiết kiệm bộ nhớ nếu file lớn
        df = pd.read_csv(p)
        df["variable"] = df["variable"].astype(str)
        # Gán nhãn y
        df.loc[df["variable"].isin(['2019', '2020']), "variable"] = 'y'
        dfs.append(df)

    df_all = pd.concat(dfs, ignore_index=True)

    # 2) Min/Max row/col → tạo hình chữ nhật
    min_row, max_row = df_all["row"].min(), df_all["row"].max()
    min_col, max_col = df_all["col"].min(), df_all["col"].max()
    n_row = int(max_row - min_row + 1)
    n_col = int(max_col - min_col + 1)

    # 3) Xác định danh sách band cần thiết (QUAN TRỌNG: Phải Sort để cố định thứ tự)
    required_bands_list = sorted(list(set(selected_features + ['y'])))
    required_bands_set = set(required_bands_list)
    print(f"-> Số lượng band bắt buộc: {len(required_bands_list)}")

    # 4) Lọc Timestamp hợp lệ (TỐI ƯU HÓA TỐC ĐỘ)
    print("[B2] Lọc timestamp đầy đủ (Vectorized Check)...")

    # Chỉ giữ lại các dòng thuộc các variable quan tâm để đếm cho nhanh
    df_check = df_all[df_all["variable"].isin(required_bands_set)]

    # Đếm số lượng variable unique trong mỗi timestamp
    # Nếu timestamp T1 có đủ 13 variable -> count sẽ là 13
    ts_counts = df_check.groupby("timestamp")["variable"].nunique()

    # Lấy ra các timestamp có số lượng variable bằng đúng số lượng yêu cầu
    valid_ts_index = ts_counts[ts_counts == len(required_bands_set)].index
    ts_valid = sorted(list(valid_ts_index))

    print(f"-> Tìm thấy {len(ts_valid)} timestamp hợp lệ.")

    # 5) Chuẩn bị dữ liệu để đổ vào Tensor
    print("[B3] Chuẩn bị index và Tensor...")

    # Tạo mapping index (Dictionary comprehension)
    t_to_idx = {t: i for i, t in enumerate(ts_valid)}
    b_to_idx = {b: i for i, b in enumerate(required_bands_list)} # Dùng list đã sort

    # Lọc dữ liệu chính thức:
    # - Chỉ lấy timestamp hợp lệ
    # - Chỉ lấy variable nằm trong required_bands (Bước này sửa lỗi Index Float)
    df_valid = df_all[
        (df_all["timestamp"].isin(ts_valid)) &
        (df_all["variable"].isin(required_bands_set))
    ].copy()

    # Map sang index (Ép kiểu int rõ ràng để tránh lỗi)
    df_valid["t_idx"] = df_valid["timestamp"].map(t_to_idx).astype(int)
    df_valid["b_idx"] = df_valid["variable"].map(b_to_idx).astype(int)
    df_valid["r_idx"] = (df_valid["row"] - min_row).astype(int)
    df_valid["c_idx"] = (df_valid["col"] - min_col).astype(int)

    # 6) Đổ dữ liệu vào Tensor (Vectorized - Không cần vòng lặp)
    print("[B4] Đổ dữ liệu vào Tensor...")
    # tensor = np.zeros((len(ts_valid), len(required_bands_list), n_row, n_col), dtype=float)

    tensor = np.full(
        (len(ts_valid), len(required_bands_list), n_row, n_col),
        -1,
        dtype=float
    )

    # Numpy Advanced Indexing: Nhanh hơn loop rất nhiều
    tensor[df_valid["t_idx"].values,
           df_valid["b_idx"].values,
           df_valid["r_idx"].values,
           df_valid["c_idx"].values] = df_valid["value"].values

    # 7) Tách X và y
    y_idx = b_to_idx['y']
    # Lấy mảng X indices: loại bỏ index của y
    x_indices = [i for i, b in enumerate(required_bands_list) if b != 'y']

    y = tensor[:, [y_idx], :, :]
    x = tensor[:, x_indices, :, :]

    return x, y, ts_valid, required_bands_list, (min_row, max_row), (min_col, max_col)

In [14]:
# # Gọi hàm y như cũ, nó sẽ tự trả về tensor 3 chiều
# list_path_csv = [OUTPUT_Y_CSV]
# tensor_3d, times, bands, row_range, col_range = csv2numpy_thanhhoa(list_path_csv)
#
# # Kiểm tra shape
# print(tensor_3d.shape)
# # Kết quả sẽ là: (Số lượng Time, Số lượng Row, Số lượng Col)